In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QComboBox, QPushButton, QVBoxLayout, QWidget, QFileDialog, QMessageBox
from PyQt5.QtGui import QPixmap, QImage
import cv2
import numpy as np
import os
from PIL import Image
import time
#from fgp_algorithm import fgp_denoise
from ADMM import ADMM
#from gd_algorithm import gd_denoise
#from pdn_algorithm import pdn_denoise
#from asd_algorithm import asd_denoise
#from fgh_algorithm import fgh_denoise

# The main application class
class ImageDenoisingApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Image Denoising App")
        self.image_path = None
        self.algorithm = None
        self.process_flag = None
        self.N = 100 # maxiter
        # Create the central widget and layout
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        # Add a label to display the image
        self.image_label = QLabel(self)
        self.layout.addWidget(self.image_label)

        # Add a combo box to select the image type
        self.image_type_combo = QComboBox(self)
        self.image_type_combo.addItem("Black and White")
        self.image_type_combo.addItem("Colour")
        self.image_type_combo.currentIndexChanged.connect(self.image_type_changed)
        self.layout.addWidget(self.image_type_combo)

        # Add a combo box to select the algorithm
        self.algorithm_combo = QComboBox(self)
        self.layout.addWidget(self.algorithm_combo)

        # Add a button to upload the image
        self.upload_button = QPushButton("Upload Image", self)
        self.upload_button.clicked.connect(self.upload_image)
        self.layout.addWidget(self.upload_button)

        # Add a button to process the image
        self.process_button = QPushButton("Process Image", self)
        self.process_button.clicked.connect(self.process_image)
        # self.process_button.clicked.connect(self.plot_image)
        self.layout.addWidget(self.process_button)

        # Add a button to show image denoising dynamics
        self.dynamics_button = QPushButton("Processing Dynamics", self)
        # self.process_button.clicked.connect(self.process_image)
        self.dynamics_button.clicked.connect(self.plot_image)
        self.layout.addWidget(self.dynamics_button)

        # Add a button to download the image
        self.download_button = QPushButton("Download Image", self)
        self.download_button.clicked.connect(self.download_image)
        self.layout.addWidget(self.download_button)

        # Update the algorithm options based on the selected image type
        self.update_algorithm_options("Black and White")

    # Update the algorithm options when the image type changes
    def image_type_changed(self, index):
        image_type = self.image_type_combo.itemText(index)
        self.update_algorithm_options(image_type)

    # Update the algorithm options based on the selected image type
    def update_algorithm_options(self, image_type):
        self.algorithm_combo.clear()
        if image_type == "Black and White":
            self.algorithm_combo.addItem("FGP")
            self.algorithm_combo.addItem("ADMM")
            self.algorithm_combo.addItem("GD")
            self.algorithm_combo.addItem("PDN")
        else:
            self.algorithm_combo.addItem("ASD")
            self.algorithm_combo.addItem("FGH")

    # Open a file dialog to upload an image
    def upload_image(self):
        file_dialog = QFileDialog(self)
        self.image_path, _ = file_dialog.getOpenFileName(self, "Upload Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if self.image_path:
            pixmap = QPixmap(self.image_path)
            self.image_label.setPixmap(pixmap.scaled(400, 400))

    # Process the uploaded image using the selected algorithm
    def process_image(self):
        self.process_flag = None
        if not self.image_path:
            QMessageBox.warning(self, "Warning", "Please upload an image first.")
            return None
        if self.image_type_combo.currentText() == "Black and White":
            image = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
            self.algorithm = self.algorithm_combo.currentText()
            if self.algorithm == "FGP":
                denoised_image = FGP(image)
            elif self.algorithm == "ADMM":
                denoised_image = ADMM(image, 10, self.N)
                self.process_flag = True # processing finished
            elif self.algorithm == "PD":
                denoised_image = chambolle_pd(image)
                self.process_flag = True
            elif self.algorithm == "SB":
                denoised_image = split_bregman(image)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return None
        else:
            self.algorithm = self.algorithm_combo.currentText()
            image = cv2.imread(self.image_path)
            if self.algorithm == "ADMM":
                denoised_image = ADMM(image)
            elif self.algorithm == "FGP":
                denoised_image, denoised_image_set = FGP(image)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return None
        # return denoised_image, denoised_image_set
    # def display_image(denoised_image):
        # Display the denoised image at a rate of 24 frames per second
        # print the denoised image 
        # 24 image per second 
        denoised_image = denoised_image.astype(np.uint8)
        cv2.imwrite("denoised_image.jpg", denoised_image)
        pixmap = QPixmap("denoised_image.jpg")
        self.image_label.setPixmap(pixmap.scaled(400, 400))
    def plot_image(self):
        for i in range(self.N):
            # Create the path to the temporary image file
            path_temp = "_".join([str(i),"temp.npy"])
            # Check if the temporary image file exists
            if os.path.exists(path_temp):
                print("displaying image", i)
                # Load the image
                deno_temp = np.load(path_temp)
                # display image
                # Convert the image to the appropriate color format
                if self.image_type_combo.currentText() == "Black and White":
                    frame = cv2.cvtColor(deno_temp, cv2.COLOR_GRAY2RGB)
                else:
                    frame = cv2.cvtColor(deno_temp, cv2.COLOR_BGR2RGB)
                # Get the height and width of the image
                h, w = deno_temp.shape[:2]
                # Calculate the bytes per line
                bytesPerLine = 3*w
                # Create a QImage object from the image data
                qimg = QImage(frame.data,w,h,bytesPerLine, QImage.Format.Format_RGB888)
                # Display the frame
                cv2.imshow("Temporary Image", frame)
                # Pause for 0.5 seconds
                cv2.waitKey(500)
                # Remove the temporary image file
                os.remove(path_temp)
        cv2.destroyAllWindows()
        # Save the denoised 
    #def save_image(denoised_image):


    # Open a file dialog to download the denoised image
    def download_image(self):
        if not self.image_path:
            QMessageBox.warning(self, "Warning", "No denoised image available.")
            return
        file_dialog = QFileDialog(self)
        file_dialog.setAcceptMode(QFileDialog.AcceptSave)
        file_path, _ = file_dialog.getSaveFileName(self, "Save Denoised Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if file_path:
            cv2.imwrite(file_path, cv2.imread("denoised_image.jpg"))
            QMessageBox.information(self, "Success", "Denoised image saved successfully.")

    # Show the application window
    def show(self):
        super().show()
        self.resize(600, 600)

 # Run the application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    image_denoising_app = ImageDenoisingApp()
    image_denoising_app.show()
    sys.exit(app.exec_())

displaying image 0
displaying image 1
displaying image 2
displaying image 3
displaying image 4
displaying image 5
displaying image 6
displaying image 7
displaying image 8
displaying image 9
displaying image 10
displaying image 11
displaying image 12
displaying image 13
displaying image 14
displaying image 15
displaying image 16
displaying image 17
displaying image 18
displaying image 19
displaying image 20
displaying image 21
displaying image 22
displaying image 23
displaying image 24
displaying image 25
displaying image 26
displaying image 27
displaying image 28
displaying image 29
displaying image 30
displaying image 31
displaying image 31
displaying image 32
displaying image 33
displaying image 34
displaying image 35
displaying image 36
displaying image 37
displaying image 38
displaying image 39
displaying image 40
displaying image 41
displaying image 42
displaying image 43
displaying image 44
displaying image 45
displaying image 46
displaying image 47
displaying image 48
displaying

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: '31_temp.npy'

SystemExit: 0

e:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
